In [1]:
import pandas as pd
import pyarrow.parquet as pq

#### LOAD CANARY PERSON SUMMARY AND NEW CANARY FILE DECEMBER 2020

In [2]:

canary_0 = '../../data/CANARY/CANARY/ACTIVITY_LOG'
canary_1 = '../../data/CANARY/CANARY/PERSON_SUMMARY'
canary_2 = '../../data/CANARY/CANARY/CANARY202101'

canary0 = pq.read_table(canary_0).to_pandas()
canary1 = pq.read_table(canary_1).to_pandas()
canary2 = pq.read_table(canary_2).to_pandas()


### Inspect both files

In [3]:
canary1.head()

PERS_ID       MBR_ID GROUPNUMBER               GROUPNAME       LOB  \
0  2638452  M0006106300       37330  Medicare Advantage 175  Medicare   
1  2706146  M0010332600       36550  Medicare Advantage 125  Medicare   
2  4122088  CD116486701       35938          365 LifePoints        LG   
3  4254340  CD127136501       35938          365 LifePoints        LG   
4  4275621  CD128714401       35938          365 LifePoints       ASO   

                                           FILE_NAME   FILE_DATE  \
0  s3://cdphp-s3-us-e-p-datalake/RAW/NATIVE/WELLT...  2020-09-07   
1  s3://cdphp-s3-us-e-p-datalake/RAW/NATIVE/WELLT...  2020-09-07   
2  s3://cdphp-s3-us-e-p-datalake/RAW/NATIVE/WELLT...  2020-09-07   
3  s3://cdphp-s3-us-e-p-datalake/RAW/NATIVE/WELLT...  2020-09-07   
4  s3://cdphp-s3-us-e-p-datalake/RAW/NATIVE/WELLT...  2020-09-07   

   N_MILESTONES_ACHIEVED REGISTER_DATE LAST_ACTIVITY_DATE  \
0                      1    2020-01-26         2020-01-26   
1                      9    2020-05-27         2020-08-31   
2                     17    2019-11-20         2020-05-29   
3                     17    2019-02-04         2019-07-24   
4                     16    2020-01-24         2020-09-01   

    LAST_COMPLETED_MILESTONE  
0              REGISTER_DATE  
1   9TH_LESSON_COMPLETE_DATE  
2  17TH_LESSON_COMPLETE_DATE  
3  17TH_LESSON_COMPLETE_DATE  
4  16TH_LESSON_COMPLETE_DATE

In [4]:
canary2.head()

PERS_ID       MBR_ID GROUPNUMBER               GROUPNAME  \
0  2638452  M0006106300       37330  Medicare Advantage 175   
1  2706146  M0010332600       36550  Medicare Advantage 125   
2  2706146  M0010332600       36550  Medicare Advantage 125   
3  2706146  M0010332600       36550  Medicare Advantage 125   
4  2706146  M0010332600       36550  Medicare Advantage 125   

                                           FILE_NAME   FILE_DATE  \
0  s3://cdphp-s3-us-e-p-datalake/RAW/NATIVE/MOVEI...  2021-01-11   
1  s3://cdphp-s3-us-e-p-datalake/RAW/NATIVE/MOVEI...  2021-01-11   
2  s3://cdphp-s3-us-e-p-datalake/RAW/NATIVE/MOVEI...  2021-01-11   
3  s3://cdphp-s3-us-e-p-datalake/RAW/NATIVE/MOVEI...  2021-01-11   
4  s3://cdphp-s3-us-e-p-datalake/RAW/NATIVE/MOVEI...  2021-01-11   

                  MILESTONE COMPLETION_DATE  
0             REGISTER_DATE      2020-01-26  
1             REGISTER_DATE      2020-05-27  
2          ORIENTATION_DATE      2020-06-08  
3  3RD_LESSON_COMPLETE_DATE      2020-06-20  
4  4TH_LESSON_COMPLETE_DATE      2020-07-02

#### Drop nan Milestones

In [5]:
canary2.dropna(axis = 0, subset = ['MILESTONE'], inplace = True)

### Inspect Unique Milestones

In [6]:
milestones = canary2.MILESTONE.unique()
milestones

array(['REGISTER_DATE', 'ORIENTATION_DATE', '3RD_LESSON_COMPLETE_DATE',
       '4TH_LESSON_COMPLETE_DATE', '5TH_LESSON_COMPLETE_DATE',
       '6TH_LESSON_COMPLETE_DATE', '7TH_LESSON_COMPLETE_DATE',
       '8TH_LESSON_COMPLETE_DATE', '9TH_LESSON_COMPLETE_DATE',
       '10TH_LESSON_COMPLETE_DATE', '11TH_LESSON_COMPLETE_DATE',
       '12TH_LESSON_COMPLETE_DATE', '13TH_LESSON_COMPLETE_DATE',
       '14TH_LESSON_COMPLETE_DATE', '15TH_LESSON_COMPLETE_DATE',
       '16TH_LESSON_COMPLETE_DATE', '17TH_LESSON_COMPLETE_DATE',
       '18TH_LESSON_COMPLETE_DATE', '19TH_LESSON_COMPLETE_DATE',
       '20TH_LESSON_COMPLETE_DATE', '21ST_LESSON_COMPLETE_DATE',
       '22ND_LESSON_COMPLETE_DATE', '23RD_LESSON_COMPLETE_DATE',
       '24TH_LESSON_COMPLETE_DATE', '25TH_LESSON_COMPLETE_DATE'],
      dtype=object)

#### Make a dictionary mapper to map Milestones to Numbers

In [7]:
mapper = {"{}TH_LESSON_COMPLETE_DATE".format(i): i for i in range(4,26)}
mapper['3RD_LESSON_COMPLETE_DATE'] = 3
mapper['REGISTER_DATE'] = 1
mapper['ORIENTATION_DATE'] = 2
mapper['23RD_LESSON_COMPLETE_DATE'] = 23
mapper['21ST_LESSON_COMPLETE_DATE'] = 21
mapper['22ND_LESSON_COMPLETE_DATE'] = 22


## Map Milestones to their corresponding number

In [8]:
canary2['MILESTONE_NUMBER'] = canary2.MILESTONE.map(mapper).astype(int)

#### Function to imitate the PERSON Summary File

In [9]:
def transform_canary(group):
    """
    Function to create an person summary file from
    a canary activity log file
    
    """
    max_mil = group.MILESTONE_NUMBER.max()
    min_mil = group.MILESTONE_NUMBER.min()
    row0 = group[group.MILESTONE_NUMBER == min_mil]
    row1 = group[group.MILESTONE_NUMBER == max_mil]
    
    if len(row0) > 1:
        row0 = pd.DataFrame([row0.iloc[0]], row0.columns)
        
    if len(row1) > 1:
        row1 =  pd.DataFrame([row1.iloc[0]], row1.columns)
        
   
    row1['N_MILESTONES_ACHIEVED'] = max_mil
    row1['LOB'] ='DN'
    row1['LAST_COMPLETED_MILESTONE'] = row1['MILESTONE']
    row1['REGISTER_DATE'] = row0['COMPLETION_DATE'].values[0]
    row1['LAST_ACTIVITY_DATE'] = row1['COMPLETION_DATE'].values[0]
    
    return row1


### Transform Canary to the Required Format

In [10]:
canary_trans = canary2.groupby('PERS_ID').apply(transform_canary)
canary_trans = canary_trans.droplevel(0).reset_index(drop = True)
canary_trans.drop(labels = ['MILESTONE', 'MILESTONE_NUMBER'], inplace = True, axis = 1)

In [11]:
canary_trans.sort_values(by = 'PERS_ID').head(3)

PERS_ID       MBR_ID GROUPNUMBER               GROUPNAME  \
0  2602482  6F995322100       35938          365 LifePoints   
1  2603072  9H994959900       37330  Medicare Advantage 175   
2  2606370  M0003719100       37330  Medicare Advantage 175   

                                           FILE_NAME   FILE_DATE  \
0  s3://cdphp-s3-us-e-p-datalake/RAW/NATIVE/MOVEI...  2021-01-11   
1  s3://cdphp-s3-us-e-p-datalake/RAW/NATIVE/MOVEI...  2021-01-11   
2  s3://cdphp-s3-us-e-p-datalake/RAW/NATIVE/MOVEI...  2021-01-11   

  COMPLETION_DATE  N_MILESTONES_ACHIEVED LOB   LAST_COMPLETED_MILESTONE  \
0      2019-08-09                      1  DN              REGISTER_DATE   
1      2020-06-29                     17  DN  17TH_LESSON_COMPLETE_DATE   
2      2021-01-06                     12  DN  12TH_LESSON_COMPLETE_DATE   

  REGISTER_DATE LAST_ACTIVITY_DATE  
0    2019-08-09         2019-08-09  
1    2020-01-01         2020-06-29  
2    2020-10-23         2021-01-06

In [12]:
canary1.sort_values(by = 'PERS_ID').head(3)

PERS_ID       MBR_ID GROUPNUMBER               GROUPNAME       LOB  \
345  2602482  6F995322100       35938          365 LifePoints        LG   
991  2603072  9H994959900       37330  Medicare Advantage 175  Medicare   
897  2619378  6K991068900       36550  Medicare Advantage 125  Medicare   

                                             FILE_NAME   FILE_DATE  \
345  s3://cdphp-s3-us-e-p-datalake/RAW/NATIVE/WELLT...  2020-09-07   
991  s3://cdphp-s3-us-e-p-datalake/RAW/NATIVE/WELLT...  2020-09-07   
897  s3://cdphp-s3-us-e-p-datalake/RAW/NATIVE/WELLT...  2020-09-07   

     N_MILESTONES_ACHIEVED REGISTER_DATE LAST_ACTIVITY_DATE  \
345                      1    2019-08-09         2019-08-09   
991                     17    2020-01-01         2020-06-29   
897                     16    2019-10-31         2020-05-01   

      LAST_COMPLETED_MILESTONE  
345              REGISTER_DATE  
991  17TH_LESSON_COMPLETE_DATE  
897  17TH_LESSON_COMPLETE_DATE

In [13]:
canary_trans.to_pickle('../../temp/CANARY.p')